In [1]:
import py_entitymatching as em
import pandas as pd
import os, sys
import pandas_profiling
import warnings
import numpy as np
import re
warnings.filterwarnings('ignore')

In [2]:
A = em.read_csv_metadata("/mnt/c/Users/sreya/Downloads/DS/bestbuy_music.csv", key= "ID")
B = em.read_csv_metadata("/mnt/c/Users/sreya/Downloads/DS/metacritic_music.csv", key= "ID")

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [3]:
em.set_key(A, 'ID')
em.set_key(B, 'ID')
em.get_property(A, 'key')

'ID'

In [4]:
G = em.read_csv_metadata("/mnt/c/Users/sreya/Downloads/DS/sampled_candidate_set.csv", key='_id',ltable=A, rtable=B, fk_ltable='ltable_ID', fk_rtable='rtable_ID')

Metadata file is not present in the given path; proceeding to read the csv file.


In [5]:
IJ = em.split_train_test(G, train_proportion=0.7, random_state=0)
I = IJ['train']
J = IJ['test']

In [6]:
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

In [7]:
F = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

In [8]:
# List the names of the features generated
F['feature_name']

0                             ID_ID_exm
1                             ID_ID_anm
2                        ID_ID_lev_dist
3                         ID_ID_lev_sim
4           Title_Title_jac_qgm_3_qgm_3
5       Title_Title_cos_dlm_dc0_dlm_dc0
6       Title_Title_jac_dlm_dc0_dlm_dc0
7                       Title_Title_mel
8                  Title_Title_lev_dist
9                   Title_Title_lev_sim
10                      Title_Title_nmw
11                       Title_Title_sw
12        Artist_Artist_jac_qgm_3_qgm_3
13    Artist_Artist_cos_dlm_dc0_dlm_dc0
14    Artist_Artist_jac_dlm_dc0_dlm_dc0
15                    Artist_Artist_mel
16               Artist_Artist_lev_dist
17                Artist_Artist_lev_sim
18                    Artist_Artist_nmw
19                     Artist_Artist_sw
20          Genre_Genre_jac_qgm_3_qgm_3
21      Genre_Genre_cos_dlm_dc0_dlm_dc0
22      Genre_Genre_jac_dlm_dc0_dlm_dc0
23                      Genre_Genre_mel
24                 Genre_Genre_lev_dist


In [9]:
H = em.extract_feature_vecs(I, 
                            feature_table=F, 
                            attrs_after='Match',
                            show_progress=False)

In [10]:
H.head(3)

,_id,ltable_ID,rtable_ID,ID_ID_exm,ID_ID_anm,ID_ID_lev_dist,ID_ID_lev_sim,Title_Title_jac_qgm_3_qgm_3,Title_Title_cos_dlm_dc0_dlm_dc0,Title_Title_jac_dlm_dc0_dlm_dc0,...,Genre_Genre_lev_sim,Genre_Genre_nmw,Genre_Genre_sw,Release_Release_lev_dist,Release_Release_lev_sim,Release_Release_jar,Release_Release_jwn,Release_Release_exm,Release_Release_jac_qgm_3_qgm_3,Match
221,31925,258,645,0,0.400000,3,0.000000,0.050633,0.134840,0.066667,...,0.375000,-2.0,3.0,8,0.2,0.474074,0.526667,0,0.095238,0
439,39218,4157,803,0,0.193168,4,0.000000,0.098361,0.166667,0.083333,...,0.214286,-8.0,3.0,9,0.0,0.412037,0.412037,0,0.000000,0
191,24496,828,523,0,0.631643,2,0.333333,0.200000,0.288675,0.166667,...,0.375000,-2.0,3.0,9,0.1,0.408333,0.408333,0,0.000000,0


In [11]:
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'Match'],
        k=5,
        target_attr='Match', metric_to_select_matcher='f1', random_state=0)
result['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,1.000000,0.876667,0.932929
1,RF,0.975000,0.888095,0.928100
2,SVM,0.400000,0.050000,0.088889
3,LinReg,0.911111,0.931429,0.918305
4,LogReg,1.000000,0.966667,0.981818
5,NaiveBayes,0.630851,1.000000,0.771439


In [12]:
C = em.read_csv_metadata("/mnt/c/Users/sreya/Downloads/candidates_large1.csv", 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_ID', fk_rtable='rtable_ID')
C

Metadata file is not present in the given path; proceeding to read the csv file.


,Unnamed: 0,_id,ltable_ID,rtable_ID,ltable_Title,ltable_Artist,ltable_Genre,ltable_Release,ltable_Rating,rtable_Title,rtable_Artist,rtable_Genre,rtable_Release,rtable_Rating
0,80,80,169,2,reputation [CD],Taylor Swift,Pop,11/10/2017,4.6,reputation,Taylor Swift,"Pop,Adult Contemporary,Pop/Rock,Dance-Pop,Teen Pop",10-Nov-17,7.7
1,81,81,912,2,reputation [Picture Disc] [2 LP] [LP] - VINYL,Taylor Swift,Pop,12/15/2017,4.0,reputation,Taylor Swift,"Pop,Adult Contemporary,Pop/Rock,Dance-Pop,Teen Pop",10-Nov-17,7.7
2,82,82,2266,3,Camila [CD],Camila Cabello,Latin,1/12/2018,4.8,Camila,Camila Cabello,"Pop,Pop/Rock,Dance-Pop,Pop Idol,Teen Pop",12-Jan-18,8.9
3,88,88,516,5,Unapologetically [CD],Kelsea Ballerini,Country,11/3/2017,4.8,Unapologetically,Kelsea Ballerini,"Pop,Country,Pop/Rock,Contemporary Country,Country-Pop",3-Nov-17,7.5
4,135,135,2304,8,The Neighbourhood [CD],The Neighbourhood,Electronic/Dance,3/9/2018,NaN,The Neighbourhood,The Neighbourhood,"Pop,Electronic,Pop/Rock,Contemporary Pop/Rock,Left-Field Pop,Synthwave",9-Mar-18,tbd
5,155,155,1318,10,Thank You [CD],Meghan Trainor,Pop,5/13/2016,4.5,Thank You,Meghan Trainor,"Pop,Pop/Rock,Dance-Pop,Teen Pop,Contemporary Pop/Rock",13-May-16,5.5
6,167,167,2188,10,Thank You [Deluxe Edition] [CD],Meghan Trainor,Pop,5/13/2016,4.5,Thank You,Meghan Trainor,"Pop,Pop/Rock,Dance-Pop,Teen Pop,Contemporary Pop/Rock",13-May-16,5.5
7,193,193,1282,12,Oh My My [CD],OneRepublic,Pop,10/7/2016,4.6,Oh My My,OneRepublic,"Pop,Pop/Rock,Alternative/Indie Rock,Contemporary Pop/Rock",7-Oct-16,7.9
8,306,306,2696,18,Mad Love [CD] [PA],JoJo,Pop,10/14/2016,4.7,Mad Love,JoJo,"Pop,Pop/Rock,Dance-Pop",14-Oct-16,8.6
9,330,330,4357,18,Mad Love [CD],JoJo,Pop,10/14/2016,5.0,Mad Love,JoJo,"Pop,Pop/Rock,Dance-Pop",14-Oct-16,8.6


In [13]:
K = em.extract_feature_vecs(C, 
                            feature_table=F,
                            show_progress=False)
# Impute feature vectors with the mean of the column values
K = em.impute_table(K, 
                exclude_attrs=['_id', 'ltable_ID', 'rtable_ID'],
                strategy='mean')

# fit with best matcher which was logistic regression
lg.fit(table=H, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'Match'], 
       target_attr='Match')
    
# take best classifier (logistic regression) and output predictions (i.e. matches)
predictions = lg.predict(table=K, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID'], 
                        append=True, target_attr='predicted', inplace=False)

# save set of matches between two tables
matches = predictions.loc[predictions['predicted'] == 1]
matches = matches.loc[:,'_id'].tolist()
mask = C['_id'].isin(matches)
mat = C.loc[mask]
mat.drop_duplicates()
#matches.csv
mat

,Unnamed: 0,_id,ltable_ID,rtable_ID,ltable_Title,ltable_Artist,ltable_Genre,ltable_Release,ltable_Rating,rtable_Title,rtable_Artist,rtable_Genre,rtable_Release,rtable_Rating
0,80,80,169,2,reputation [CD],Taylor Swift,Pop,11/10/2017,4.6,reputation,Taylor Swift,"Pop,Adult Contemporary,Pop/Rock,Dance-Pop,Teen Pop",10-Nov-17,7.7
1,81,81,912,2,reputation [Picture Disc] [2 LP] [LP] - VINYL,Taylor Swift,Pop,12/15/2017,4.0,reputation,Taylor Swift,"Pop,Adult Contemporary,Pop/Rock,Dance-Pop,Teen Pop",10-Nov-17,7.7
2,82,82,2266,3,Camila [CD],Camila Cabello,Latin,1/12/2018,4.8,Camila,Camila Cabello,"Pop,Pop/Rock,Dance-Pop,Pop Idol,Teen Pop",12-Jan-18,8.9
3,88,88,516,5,Unapologetically [CD],Kelsea Ballerini,Country,11/3/2017,4.8,Unapologetically,Kelsea Ballerini,"Pop,Country,Pop/Rock,Contemporary Country,Country-Pop",3-Nov-17,7.5
4,135,135,2304,8,The Neighbourhood [CD],The Neighbourhood,Electronic/Dance,3/9/2018,NaN,The Neighbourhood,The Neighbourhood,"Pop,Electronic,Pop/Rock,Contemporary Pop/Rock,Left-Field Pop,Synthwave",9-Mar-18,tbd
5,155,155,1318,10,Thank You [CD],Meghan Trainor,Pop,5/13/2016,4.5,Thank You,Meghan Trainor,"Pop,Pop/Rock,Dance-Pop,Teen Pop,Contemporary Pop/Rock",13-May-16,5.5
6,167,167,2188,10,Thank You [Deluxe Edition] [CD],Meghan Trainor,Pop,5/13/2016,4.5,Thank You,Meghan Trainor,"Pop,Pop/Rock,Dance-Pop,Teen Pop,Contemporary Pop/Rock",13-May-16,5.5
7,193,193,1282,12,Oh My My [CD],OneRepublic,Pop,10/7/2016,4.6,Oh My My,OneRepublic,"Pop,Pop/Rock,Alternative/Indie Rock,Contemporary Pop/Rock",7-Oct-16,7.9
8,306,306,2696,18,Mad Love [CD] [PA],JoJo,Pop,10/14/2016,4.7,Mad Love,JoJo,"Pop,Pop/Rock,Dance-Pop",14-Oct-16,8.6
9,330,330,4357,18,Mad Love [CD],JoJo,Pop,10/14/2016,5.0,Mad Love,JoJo,"Pop,Pop/Rock,Dance-Pop",14-Oct-16,8.6


In [14]:
#Splitting the indices of ltable(Metacritic) and rtable(BestBuy)
metacriticMatchInd = [int(s)-1 for s in list(predictions.loc[predictions['predicted'] == 1,'ltable_ID'])]
BBMatchInd = [int(s)-1 for s in list(predictions.loc[predictions['predicted'] == 1,'rtable_ID'])]

In [15]:
E = pd.DataFrame(columns = A.columns[1:])

In [16]:
#merging rtable and ltable values to get E
def mergeTuples(x1, x2):
    t = []
    for i in np.arange(1,len(x1)):
        if A.columns[i] in ['Title','Artist','Genre']:
            if len(x1[i]) >= len(x2[i]):
                curr = x1[i]
                t.append(curr)
            else:
                curr = x2[i]
                t.append(curr)
        if A.columns[i] in ['Release']:
            t.append(x2[i])
        if A.columns[i] in ['Rating']:
            if(x1[i] == 'tbd'):
                t.append(0.0)
            else:
                t.append(float(x1[i]))
    return t

In [17]:
#Calling mergetuples on tuples from BestBuy and Metacritic
for i in np.arange(0,len(BBMatchInd)):
    BBTuple = B.loc[BBMatchInd[i]][0:]
    #print(BBTuple)
    metaTuple = A.loc[metacriticMatchInd[i]][0:]
    #print(metaTuple)
    merged = mergeTuples(BBTuple, metaTuple)
    E.loc[i] = merged

In [18]:
#E Merged Table
E

,Title,Artist,Genre,Release,Rating
0,reputation [CD],Taylor Swift,"Pop,Adult Contemporary,Pop/Rock,Dance-Pop,Teen Pop",11/10/2017,7.7
1,reputation [Picture Disc] [2 LP] [LP] - VINYL,Taylor Swift,"Pop,Adult Contemporary,Pop/Rock,Dance-Pop,Teen Pop",12/15/2017,7.7
2,Camila [CD],Camila Cabello,"Pop,Pop/Rock,Dance-Pop,Pop Idol,Teen Pop",1/12/2018,8.9
3,Unapologetically [CD],Kelsea Ballerini,"Pop,Country,Pop/Rock,Contemporary Country,Country-Pop",11/3/2017,7.5
4,The Neighbourhood [CD],The Neighbourhood,"Pop,Electronic,Pop/Rock,Contemporary Pop/Rock,Left-Field Pop,Synthwave",3/9/2018,0.0
5,Thank You [CD],Meghan Trainor,"Pop,Pop/Rock,Dance-Pop,Teen Pop,Contemporary Pop/Rock",5/13/2016,5.5
6,Thank You [Deluxe Edition] [CD],Meghan Trainor,"Pop,Pop/Rock,Dance-Pop,Teen Pop,Contemporary Pop/Rock",5/13/2016,5.5
7,Oh My My [CD],OneRepublic,"Pop,Pop/Rock,Alternative/Indie Rock,Contemporary Pop/Rock",10/7/2016,7.9
8,Mad Love [CD] [PA],JoJo,"Pop,Pop/Rock,Dance-Pop",10/14/2016,8.6
9,Mad Love [CD],JoJo,"Pop,Pop/Rock,Dance-Pop",10/14/2016,8.6


In [19]:
#Getting the year from Release date
D = E
arr = D['Release'].str.split('/').values
D['Release'] = [s[2] for s in arr]

In [20]:
df_2017 = D.groupby(['Release']).get_group('2017')
df_2017

,Title,Artist,Genre,Release,Rating
0,reputation [CD],Taylor Swift,"Pop,Adult Contemporary,Pop/Rock,Dance-Pop,Teen Pop",2017,7.7
1,reputation [Picture Disc] [2 LP] [LP] - VINYL,Taylor Swift,"Pop,Adult Contemporary,Pop/Rock,Dance-Pop,Teen Pop",2017,7.7
3,Unapologetically [CD],Kelsea Ballerini,"Pop,Country,Pop/Rock,Contemporary Country,Country-Pop",2017,7.5
10,So Good [CD] [PA],Zara Larsson,"Pop,Electronic,Pop/Rock,Dance-Pop",2017,7.6
38,The Great Gatsby [Music from Baz Luhrmann's Film],Various Artists,"Pop,Rap,Avant-Garde,Pop/Rock,Adult Alternative Pop/Rock,Alternative Pop/Rock,Contemporary Pop/Ro...",2017,7.8
39,The Great Gatsby [Music from Baz Luhrmann's Film],Various Artists,"Pop,Rap,Avant-Garde,Pop/Rock,Adult Alternative Pop/Rock,Alternative Pop/Rock,Contemporary Pop/Ro...",2017,7.8
40,The Great Gatsby [Music from Baz Luhrmann's Film],Various Artists,"Pop,Rap,Avant-Garde,Pop/Rock,Adult Alternative Pop/Rock,Alternative Pop/Rock,Contemporary Pop/Ro...",2017,7.8
58,The Truth About Love,P!nk,"Pop,Pop/Rock,Dance-Pop,Contemporary Pop/Rock",2017,8.2
69,After Laughter [CD],Paramore,"Pop,Pop/Rock,Alternative/Indie Rock,Emo-Pop,Left-Field Pop",2017,8.2
70,After Laughter [LP] - VINYL,Paramore,"Pop,Pop/Rock,Alternative/Indie Rock,Emo-Pop,Left-Field Pop",2017,8.2


In [21]:
#Testing on year 2017
arr1 = df_2017['Genre'].str.split(',').values
lst = []
for val in arr1:
    if any("Rock" in s for s in val):
        if any("Contemporary" in s for s in val):
            lst.append("Contemporary Rock")
        elif any("Alternative" in s for s in val):
            lst.append("Alternative Rock")
        else:
            lst.append("Rock")
    else:
        lst.append('Pop')  
        
df_2017['Genre']  = lst
df_2017

,Title,Artist,Genre,Release,Rating
0,reputation [CD],Taylor Swift,Contemporary Rock,2017,7.7
1,reputation [Picture Disc] [2 LP] [LP] - VINYL,Taylor Swift,Contemporary Rock,2017,7.7
3,Unapologetically [CD],Kelsea Ballerini,Contemporary Rock,2017,7.5
10,So Good [CD] [PA],Zara Larsson,Rock,2017,7.6
38,The Great Gatsby [Music from Baz Luhrmann's Film],Various Artists,Contemporary Rock,2017,7.8
39,The Great Gatsby [Music from Baz Luhrmann's Film],Various Artists,Contemporary Rock,2017,7.8
40,The Great Gatsby [Music from Baz Luhrmann's Film],Various Artists,Contemporary Rock,2017,7.8
58,The Truth About Love,P!nk,Contemporary Rock,2017,8.2
69,After Laughter [CD],Paramore,Alternative Rock,2017,8.2
70,After Laughter [LP] - VINYL,Paramore,Alternative Rock,2017,8.2


In [22]:
meanrating = df_2017.groupby('Genre')['Rating'].agg(['count','mean']).reset_index()
meanrating

,Genre,count,mean
0,Alternative Rock,2,8.200000
1,Contemporary Rock,12,7.741667
2,Pop,2,7.150000
3,Rock,4,7.550000


In [23]:
#Afer grouping by year we segregate into different genres (Rock, Alternative, Contemporary and Pop)
def find_Rock(name, group):
    arr1 = group['Genre'].str.split(',').values
    lst = []
    for val in arr1:
        if any("Rock" in s for s in val):
            if any("Contemporary" in s for s in val):
                lst.append("Contemporary")
            elif any("Alternative" in s for s in val):
                lst.append("Alternative")
            else:
                lst.append("Rock")
        else:
            lst.append('Pop')  
    group['Genre']  = lst
    print(name)
    print(group.groupby('Genre')['Rating'].agg(['count','mean']).reset_index())
    print()

grouped = D.groupby(['Release'])
for name, group in grouped:
    find_Rock(name, group)

2000
  Genre  count  mean
0   Pop      1   8.5

2001
          Genre  count  mean
0   Alternative      1  8.80
1  Contemporary      2  7.65
2           Pop      1  7.50

2002
          Genre  count  mean
0   Alternative      1   8.9
1  Contemporary      1   7.8
2           Pop      2   8.1
3          Rock      1   8.2

2003
  Genre  count  mean
0   Pop      1   8.1

2004
          Genre  count  mean
0  Contemporary      1   7.8

2005
          Genre  count  mean
0   Alternative      1   9.0
1  Contemporary      1   6.2
2           Pop      1   6.0

2006
          Genre  count  mean
0  Contemporary      1  7.40
1           Pop      2  7.35

2007
          Genre  count  mean
0   Alternative      1   8.1
1  Contemporary      2   7.8
2          Rock      1   7.8

2008
         Genre  count    mean
0  Alternative      2  8.6500
1         Rock      8  8.1875

2009
  Genre  count      mean
0   Pop      1  7.700000
1  Rock      3  8.266667

2010
         Genre  count  mean
0  Alternative      